In [1]:
import pickle
import torch
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from pathlib import Path
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from modules.action_recognizer.dataset.pose_dataset import PoseDataset, all_activities
from modules.action_recognizer.models.action_transformer.lit_action_transformer import LitActionTransformer
import shutil

pl.seed_everything(1234)

Seed set to 1234


1234

In [2]:
with open('output/inner_mirror_with_padding/pose_info.pkl', 'rb') as f:
    pose_df = pickle.load(f)

with open('output/inner_mirror_with_padding/annotation.pkl', 'rb') as f:
    annotation_df = pickle.load(f)

In [3]:
from modules.definition import bone_connections

In [49]:
bone_list = []
velocity_list = []
for index, row in annotation_df.iterrows():
    start, end = row.frame_index_start, row.frame_index_end
    frame_seq = pose_df.loc[start: end - 1]
    pose_3d_list = frame_seq['pose_3d']

    pose_3d = np.stack(pose_3d_list.values)
    bones = []
    for bone_connection in bone_connections:
        kp_from, kp_to = bone_connection
        bone = pose_3d[:, kp_to - 1, :] - pose_3d[:, kp_from - 1, :]
        bones.append(bone)
    bone_data = np.stack(bones, axis=1)
    velocity = np.zeros_like(pose_3d)
    velocity[:-1] = pose_3d[1:] - pose_3d[:-1]
    velocity_list += velocity.tolist()
    bone_list += bone_data.tolist()
pose_df = pose_df.assign(bone=pd.Series(bone_list))
pose_df = pose_df.assign(velocity=pd.Series(velocity_list))

In [50]:
pose_df.to_pickle('output/inner_mirror_with_padding/new_pose_info.pkl')